In [1]:
%load_ext Cython

In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.ensemble as ske
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV , train_test_split
from stroopwafel import constants , utils
from sklearn import metrics , tree
import seaborn as sns
from numpy import array as A
from sklearn.utils import shuffle
import cython
from cython.parallel import prange , parallel
from cython import nogil
import random
from sklearn.preprocessing import OneHotEncoder
import pickle
from mpl_toolkits import mplot3d

In [3]:
%%cython

# cython: auto_pickle=False
# distutils: extra_compile_args = /openmp
# distutils: extra_link_args = /openmp
# cython: boundscheck = False

import cython
import numpy as np
cimport numpy as np
cimport openmp
from cython.parallel import prange, parallel, threadid
from libc.stdlib cimport malloc, realloc, free, abort
from libc.math cimport cos , sin , sqrt , tan

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)

cdef Hot_Encode(int L , int n_class , int n_var , double [:,:] dataset , int [:,:] labels , int num_t):
    
    cdef double [:,:] DATASET = dataset[0:L,:]  
    cdef int [:,:] LABELS = labels[0:L,:]
    cdef int i , j
    cdef int [:] merge = labels[:,0]
    cdef int [:] bbh = labels[:,1]
    cdef double [:,:,:] ds
    
    if n_class >= n_var:
        ds = np.zeros( (2 , L , n_class) )
    else:
        ds = np.zeros( (2 , L , n_var) )

    for i in prange(L , nogil = True , schedule = 'static' , num_threads = num_t):
        
        for j in range(n_var):
            ds[0,i,j] = DATASET[i,j]
        
        if merge[i] == 1 and bbh[i] == 1: # BBH MERGER
            
            if labels[i,2] != 0 and labels[i,3] != 0: # BBH MERGER WITH DOUBLE CE
                ds[1,i,0] = 1
            elif labels[i,2] != 0 and labels[i,3] == 0: # BBH MERGER WITH SINGLE CE
                ds[1,i,0] = 1
            elif labels[i,2] == 0 and labels[i,3] != 0: # BBH MERGER WITH SINGLE CE
                ds[1,i,0] = 1
                
            else: # BBH MERGER WITH NO CE
                ds[1,i,1] = 1
                
        elif merge[i] == 0 and bbh[i] == 1: # BBH NO MERGER:
            
            if labels[i,2] != 0 and labels[i,3] != 0: # BBH NO MERGER WITH DOUBLE CE
                ds[1,i,2] = 1
            elif labels[i,2] != 0 and labels[i,3] == 0: # BBH NO MERGER WITH SINGLE CE
                ds[1,i,2] = 1
            elif labels[i,2] == 0 and labels[i,3] != 0: # BBH NO MERGER WITH SINGLE CE
                ds[1,i,2] = 1
                
            else: # BBH NO MERGER NO CE
                ds[1,i,3] = 1
                
        elif merge[i] == 0 and bbh[i] == 0: # NO BBH NO MERGER
            ds[1,i,4] = 1
        
    return ds

def hotencode(int L , int n_class , int n_var , double [:,:] dataset , int [:,:] labels , int num_t):
    return Hot_Encode(L , n_class , n_var , dataset , labels , num_t)

print('hotencoding ok!')

hotencoding ok!


In [4]:
def Equalize(variable , label , minimum , n , L):
    
    lab = []
    var = []
    
    for i in range(L):
        if label.iloc[i,1] == 1:
            lab.append(label.iloc[i].values)
            var.append(variable.iloc[i].values)

    for N in range(n):
        k = 0
        for j in range(L):
            if label.iloc[j,N] == 1 and k <= minimum-1 and N != 1:
                lab.append(label.iloc[j].values)
                var.append(variable.iloc[j].values)
                k += 1
                
    return np.array(lab) , np.array(var)

print('Equalize ok!')

Equalize ok!


In [5]:
dataset = pd.read_csv('SAMPLES_TRUE.csv')
print(len(dataset))

VAR = dataset[['--initial-mass-1' , 'q' , '--metallicity' , '--semi-major-axis' , '--eccentricity']]
LAB = dataset[['is_hit' , 'bbh' , 'time_common_enveloe_1' , 'time_common_enveloe_2']].astype(int)

n = 5
m = 5
threads = 8
L = 1000000#int(len(dataset))

ds = hotencode(L , n , m , VAR.values , LAB.astype(int).values , threads)

variables = np.array(ds[0,:,0:m])
labels = np.array(ds[1,:,0:n]).astype(int)

names = ['BBH_MERGE_CE' , 'BBH_MERGE' , 'BBH_CE' , 'BBH' , 'NOTHING']

c = 0
for i in range(n):
    c += 100*(np.count_nonzero(labels[:,i] == 1)/len(labels[:,i]))
    print(names[i] ,  '{:.2f}..'.format(100*(np.count_nonzero(labels[:,i] == 1)/len(labels[:,i]))) , '% ---->' , np.count_nonzero(labels[:,i] == 1))

if c >= 99.9:
    print('')
    print('OK! All dataset is represented!')
    
counts = np.zeros(n)
for i in range(n):
    counts[i] = np.count_nonzero(labels[:,i] == 1)
minimum = int(np.min(counts))

print('')
print('Minimum = ' , minimum)
print('')

LABELS = pd.DataFrame(labels , columns = names , dtype = int)
VARIABLES = pd.DataFrame(variables , columns = ['M1' , 'q' , 'Z' , 'a' , 'e'])

lab , var = Equalize(VARIABLES , LABELS , minimum , n , L)

for i in range(n):
    print(names[i] ,  '{:.2f}..'.format(100*(np.count_nonzero(lab[:,i] == 1)/len(lab[:,i]))) , '% ---->' , np.count_nonzero(lab[:,i] == 1))

1500000
BBH_MERGE_CE 2.05.. % ----> 20508
BBH_MERGE 0.39.. % ----> 3875
BBH_CE 0.83.. % ----> 8293
BBH 2.85.. % ----> 28513
NOTHING 93.88.. % ----> 938811

OK! All dataset is represented!

Minimum =  3875

BBH_MERGE_CE 20.00.. % ----> 3875
BBH_MERGE 20.00.. % ----> 3875
BBH_CE 20.00.. % ----> 3875
BBH 20.00.. % ----> 3875
NOTHING 20.00.. % ----> 3875


In [6]:
datasetX = var

datasetY = lab

Xtrain, Xtest, ytrain, ytest = train_test_split( datasetX , datasetY , test_size = 0.2 , random_state = 311996 )

print(len(Xtrain) , len(ytrain) , len(Xtest) , len(ytest))

15500 15500 3875 3875


In [7]:
# Number of trees in random forest
n_estimators = np.arange(100 , 5100 , 100).tolist()

# Number of features to consider at every split
max_features = ['sqrt' , 'log2' , 'auto']

# Maximum number of levels in tree
max_depth = np.arange(10,1050,50 , dtype = int).tolist()
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = np.arange(2,11,1).tolist()

# Minimum number of samples required at each leaf node
min_samples_leaf = np.arange(2,11,1).tolist()

# Method of selecting samples for training each tree
bootstrap = [True , False]

criterion = ["gini" , "entropy"]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}
print(random_grid)

# Use the random grid to search for best hyperparameters

# Random search of parameters, using 5 fold cross validation, 
# search across 60 different combinations, and use all available cores

rf_random = RandomizedSearchCV( estimator = ske.RandomForestClassifier(n_jobs = -1) , 
                                param_distributions = random_grid , n_iter = 60 ,
                                cv = 5, verbose = 2 , random_state = 3117102616 , n_jobs = -1 , pre_dispatch = '2*n_jobs' ) 

# Fit the random search model
rf_random.fit(Xtrain , ytrain)

print(rf_random.best_params_)

print('Best accuracy score: %.4f' , rf_random.best_score_)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000], 'max_features': ['sqrt', 'log2', 'auto'], 'max_depth': [10, 60, 110, 160, 210, 260, 310, 360, 410, 460, 510, 560, 610, 660, 710, 760, 810, 860, 910, 960, 1010, None], 'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']}
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 32.4min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 136.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 273.6min finished


{'n_estimators': 3700, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 310, 'criterion': 'entropy', 'bootstrap': False}
Best accuracy score: %.4f 0.833483870967742


In [8]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [260 , 270 , 280 , 290 , 300 , 310 , 320 , 330 , 340 , 350 , 360],
    'n_estimators': [3600 , 3625 , 3650 , 3675 , 3700 , 3725 , 3750 , 3775 , 3800],
    'min_samples_leaf': [2],
    'min_samples_split': [8,9,10]
}
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = ske.RandomForestClassifier(max_features = 'auto',
                                                                  bootstrap = False , criterion = "entropy",
                                                                  n_jobs = -1),
                           param_grid = param_grid, 
                           cv = 3, n_jobs = -1, verbose = 2 , pre_dispatch = '2*n_jobs')

grid_search.fit(Xtrain , ytrain)

print(grid_search.best_params_)
print('Best accuracy score: %.4f' , grid_search.best_score_)

Fitting 3 folds for each of 297 candidates, totalling 891 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 50.5min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 248.0min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 579.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 1036.3min
[Parallel(n_jobs=-1)]: Done 891 out of 891 | elapsed: 2082.6min finished


{'max_depth': 300, 'min_samples_leaf': 2, 'min_samples_split': 8, 'n_estimators': 3700}
Best accuracy score: %.4f 0.8281291806807863


In [12]:
dataset = pd.read_csv('SAMPLES_TRUE.csv')

VAR = dataset[['--initial-mass-1' , 'q' , '--metallicity' , '--semi-major-axis' , '--eccentricity']]
LAB = dataset[['is_hit' , 'bbh' , 'time_common_enveloe_1' , 'time_common_enveloe_2']].astype(int)
n = 5
m = 5
threads = 8
L = len(dataset)

ds = hotencode(L , n , m , VAR.values , LAB.astype(int).values , threads)

variables = np.array(ds[0,:,0:m])
labels = np.array(ds[1,:,0:n]).astype(int)

names = ['BBH_MERGE_CE' , 'BBH_MERGE' , 'BBH_CE' , 'BBH' , 'NOTHING']

c = 0
for i in range(n):
    c += 100*(np.count_nonzero(labels[:,i] == 1)/len(labels[:,i]))
    print(names[i] ,  '{:.2f}..'.format(100*(np.count_nonzero(labels[:,i] == 1)/len(labels[:,i]))) , '% ---->' , np.count_nonzero(labels[:,i] == 1))

if c >= 99.9:
    print('')
    print('OK! All the dataset is represented!')
    
counts = np.zeros(n)
for i in range(n):
    counts[i] = np.count_nonzero(labels[:,i] == 1)
minimum = int(np.min(counts))

print('')
print('Minimum = ' , minimum)
print('')

LABELS = pd.DataFrame(labels , columns = names , dtype = int)
VARIABLES = pd.DataFrame(variables , columns = ['M1' , 'q' , 'Z' , 'a' , 'e'])

lab , var = Equalize(VARIABLES , LABELS , minimum , n , L)

for i in range(n):
    print(names[i] ,  '{:.2f}..'.format(100*(np.count_nonzero(lab[:,i] == 1)/len(lab[:,i]))) , '% ---->' , np.count_nonzero(lab[:,i] == 1))

datasetX = var

datasetY = lab

Xtrain, Xtest, ytrain, ytest = train_test_split( datasetX , datasetY ,
                                                 test_size = 0.2 , random_state = 754863321 )

print(len(Xtrain) , len(ytrain) , len(Xtest) , len(ytest))

BBH_MERGE_CE 3.54.. % ----> 53025
BBH_MERGE 0.67.. % ----> 10021
BBH_CE 1.42.. % ----> 21298
BBH 4.65.. % ----> 69708
NOTHING 89.73.. % ----> 1345948

OK! All the dataset is represented!

Minimum =  10021

BBH_MERGE_CE 20.00.. % ----> 10021
BBH_MERGE 20.00.. % ----> 10021
BBH_CE 20.00.. % ----> 10021
BBH 20.00.. % ----> 10021
NOTHING 20.00.. % ----> 10021
40084 40084 10021 10021


In [13]:
final_model = ske.RandomForestClassifier(n_estimators = 3700 , min_samples_split = 8 ,
                                         min_samples_leaf = 2 ,
                                         max_features = 'auto' , max_depth = 300 ,
                                         bootstrap = False , criterion = "entropy" , n_jobs = -1)
final_model.fit(Xtrain , ytrain)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=300,
                       min_samples_leaf=2, min_samples_split=8,
                       n_estimators=3700, n_jobs=-1)

In [14]:
y_pred = final_model.predict(Xtest)

print('Mean Absolute Error:', metrics.mean_absolute_error(ytest, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(ytest, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest, y_pred)))
print("Accuracy:",metrics.accuracy_score(ytest, y_pred))


Mean Absolute Error: 0.050094800918072044
Mean Squared Error: 0.050094800918072044
Root Mean Squared Error: 0.22381867866215288
Accuracy: 0.8407344576389582


In [128]:
fig = plt.figure(figsize = [20,10])
feature_imp = pd.Series(final_model.feature_importances_ , index=['M$_{1}$' , 'q' , 'Z' , 'a' , 'e']).sort_values(ascending=False)
print(feature_imp)

sns.set(font_scale=2)

# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)

# Add labels to your graph
plt.xlabel('Feature Importance Score' , fontsize = 20)
plt.ylabel('Features' , fontsize = 20)
plt.title("Visualizing Important Features" , fontsize = 20)
plt.show()

fig.savefig('importance.png')

M$_{1}$    0.292147
a          0.265872
Z          0.185761
q          0.152348
e          0.103872
dtype: float64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [129]:
# Confusion matrix
# Evaluate confusion matrix

cm = metrics.confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1))

##############################################################################

fig = plt.figure(figsize = [10,10])

ax = sns.heatmap(cm/np.sum(cm), annot=True, fmt='.1%', cmap='Blues')

ax.set_xticklabels(names , fontsize = 10)
ax.set_yticklabels(names , fontsize = 10)

ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

plt.show()
fig.savefig('conf_matrix_perc.png')

##############################################################################

fig = plt.figure(figsize = [10,10])
ax = sns.heatmap(cm, annot=True, fmt='1', cmap='Blues')

ax.set_xticklabels(names , fontsize = 10)
ax.set_yticklabels(names , fontsize = 10)

ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ')

plt.show()

fig.savefig('conf_matrix_abs.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
filename = 'RandomForestBBH_FINALE.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [ ]:
fn = ['M1' , 'q' , 'Z' , 'a' , 'e']
cn = names

ext = final_model.estimators_[0]

sns.reset_defaults()

fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (100,100), dpi=500)
out = tree.plot_tree(ext,
               feature_names = fn, 
               class_names=cn,
               filled = True)

for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor('black')
        arrow.set_linewidth(1)

fig.savefig('rf_individualtree.png')

In [ ]:
# FAI LA PCA E PLOTTA TUTTO IN 2 D e 3 D, POI FAI IL CLUSTERING
sns.reset_defaults()
colors = [str('b') , str('r') , str('g') , str('c') , str('y')]

fig = plt.figure(figsize = [10,5])
for i in range(len(var[:,0])):
    a = lab[i]
    if np.all(a==[1,0,0,0,0]):
        plt.plot(var[i,0] , var[i,3] , colors[0]+'.' , markersize = 5)
    elif np.all(a==[0,1,0,0,0]):
        plt.plot(var[i,0] , var[i,3] , colors[1]+'.' , markersize = 5)
    elif np.all(a==[0,0,1,0,0]):
        plt.plot(var[i,0] , var[i,3] , colors[2]+'.' , markersize = 5)
    elif np.all(a==[0,0,0,1,0]):
        plt.plot(var[i,0] , var[i,3] , colors[3]+'.' , markersize = 5)
    elif np.all(a==[0,0,0,0,1]):
        plt.plot(var[i,0] , var[i,3] , colors[4]+'.' , markersize = 5)
plt.xlabel('M$_{1} [M_{\odot}]$' , fontsize  = 20)
plt.ylabel('a [AU]' , fontsize  = 20)
for j in range(len(colors)):
    plt.plot(0,0,colors[j]+'-',markersize = 0 , label = names[j])
plt.legend()
plt.show()

fig.savefig('PCA_2dim.png')

In [ ]:

fig = plt.figure()
sns.reset_defaults()
colors = [str('b') , str('r') , str('g') , str('c') , str('y')]

ax = plt.axes(projection='3d')
for i in range(len(var[:,0])):
    a = lab[i]
    if np.all(a==[1,0,0,0,0]):
        ax.scatter3D(var[i,0] , var[i,3] , var[i,2] , colors[0]+'.' , s = 5)
    elif np.all(a==[0,1,0,0,0]):
        ax.scatter3D(var[i,0] , var[i,3] , var[i,2] , colors[1]+'.' , s = 5)
    elif np.all(a==[0,0,1,0,0]):
        ax.scatter3D(var[i,0] , var[i,3] , var[i,2] , colors[2]+'.' , s = 5)
    elif np.all(a==[0,0,0,1,0]):
        ax.scatter3D(var[i,0] , var[i,3] , var[i,2] , colors[3]+'.' , s = 5)
    elif np.all(a==[0,0,0,0,1]):
        ax.scatter3D(var[i,0] , var[i,3] , var[i,2] , colors[4]+'.' , s = 5)
ax.set_xlabel('M$_{1} [M_{\odot}]$' , fontsize  = 20)
ax.set_ylabel('a [AU]' , fontsize  = 20)
ax.set_zlabel('Z [$\frac{Fe}{H}$]' , fontsize = 20)
for j in range(len(colors)):
    plt.plot(0,0,colors[j]+'-',markersize = 0 , label = names[j])
plt.legend()
plt.show()

fig.savefig('PCA_3dim.png')